In [1]:
import math
import wave
import struct
import pdb
import numpy as np
import matplotlib.pyplot as plt
import csv
import scipy.stats
import random

In [2]:
"""
Variable audio will contain a long list of samples (i.e. floating point numbers describing the
waveform).  If you were working with a very long sound you'd want to stream this to
disk instead of buffering it all in memory list this.  But most sounds will fit in 
memory.   
"""

def append_silence(audio, sample_rate, duration_milliseconds=225):
    """
    Adding silence is easy - we add zeros to the end of our array
    """
    num_samples = duration_milliseconds * (sample_rate / 1000.0)

    for x in range(int(num_samples)): 
        audio.append(0.0)
    return audio

def append_noise(audio, sample_rate, 
                 duration_milliseconds=12000, 
                 volume=1.0):
    """
    The sine wave generated here is the standard beep.  If you want something
    more aggresive you could try a square or saw tooth waveform.   Though there
    are some rather complicated issues with making high quality square and
    sawtooth waves... which we won't address here
    """ 
    num_samples = duration_milliseconds * (sample_rate / 1000.0)
    signal = []
    
    for x in range(int(num_samples)):
        signal.append(volume * np.random.normal())
    signal /= np.max(np.abs(signal))    
    signal = envelopeKCW(signal, 20, sample_rate) 
    for x_signal in range(len(signal)):
        audio.append(signal[x_signal])
    return audio

def envelopeKCW(signal,rampDur,sample_rate):
    """
    This function tries to remove the transients in the signal by enveloping the first and last period.
    Ramp duration is defined by rampDur in **ms** envelope(signal,rampDuration,samplerate)
    """
    samples = int(np.round((rampDur/1000)*sample_rate))
    x = np.arange(-np.pi,0,np.pi/samples)
    y = np.arange(0,np.pi,np.pi/samples)

    # Prepare the envelope functions
    envelope_function = np.cos(x)/2+0.5;
    
    # Fade in
    for i in range(samples):
        signal[i] = signal[i]*envelope_function[i];

    # Fade out
    for i in range(samples-1):
        current_position = len(signal)-i-1;
        signal[current_position] = signal[current_position] * envelope_function[i+1];
    return signal


def save_wav(audio, sample_rate, file_name):
    # Open up a wav file
    wav_file=wave.open(file_name,"w")

    # wav params
    nchannels = 1
    sampwidth = 2

    # 44100 is the industry standard sample rate - CD quality.  If you need to
    # save on file size you can adjust it downwards. The standard for low quality
    # is 8000 or 8kHz.
    nframes = len(audio)
    comptype = "NONE"
    compname = "not compressed"
    wav_file.setparams((nchannels, sampwidth, sample_rate, nframes, comptype, compname))

    # WAV files here are using short, 16 bit, signed integers for the 
    # sample size.  So we multiply the floating point data we have by 32767, the
    # maximum value for a short integer.  NOTE: It is theortically possible to
    # use the floating point -1.0 to 1.0 data directly in a WAV file but not
    # obvious how to do that using the wave module in python.
    pdb.set_trace()
    for sample in audio:
        wav_file.writeframes(struct.pack('h', int( sample * 32767.0 )))
        
    wav_file.close()
    return

def generate_tone_cloud():
    audio = []
    sample_rate = 44100
    
    audio = append_noise(audio = audio, sample_rate = sample_rate)
    return audio, sample_rate

In [3]:
# Generate noise stimuli
audio, sample_rate = generate_tone_cloud()        
save_wav(audio, sample_rate, file_name = "../auditory_categorization/assets/noise_headphone_check"+".wav")


> <ipython-input-2-ee169790dc42>(83)save_wav()
-> for sample in audio:
(Pdb) continue
